In [1]:
!pip install --progress-bar on transformers accelerate sentencepiece
!pip install --progress-bar on pandas numpy matplotlib
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124 -v



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cu121



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
import torch
print(torch.__version__)
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


2.6.0+cu124
Torch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
Device: NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
!pip install ipywidgets



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:

import numpy as np
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [3]:
train=pd.read_csv("CJPE_ext_SCI_HCs_Tribunals_daily_orders_single.csv")


In [4]:
train.head()

,filename,text,label
0,Calcutta_High_Court_Appellete_Side_2008_2020_2...,"Judgment on 18th April, 2008.Bhaskar J.These t...",0
1,Bombay_HC_BomHC_1937_179,"Beaumont, C.J.This is an appeal from a decisio...",0
2,karnataka_HC_1998_425,The appellant has filed this appeal assailing ...,0
3,Bombay_HC_BomHC_1994_60,"The petitioner, Forbes Forbes Campbel and.Comp...",1
4,Madras_HC_2017_4649,The writ petitioner was allotted plot No.60 fo...,1


In [5]:
val=pd.read_csv("CJPE_ext_SCI_HCs_Tribunals_daily_orders_test.csv")

In [6]:
test=pd.read_csv("CJPE_ext_SCI_HCs_Tribunals_daily_orders_dev.csv")

In [7]:
val.drop(columns=["filename"],inplace=True)
test.drop(columns=["filename"],inplace=True)
train.drop(columns=["filename"],inplace=True)

In [8]:
val.head()

,text,label
0,The plaintiffs are the appellants against the ...,1
1,(Judgment of the Court was made by The present...,0
2,"Mr. E.I.Sanmathi, Adv.for Ms. Vani H, Adv.for ...",0
3,"Govindasamy J. Lakshmi Vilas Bank, the first r...",0
4,The 1st petitioner was an LPG distributor at K...,0


In [9]:

train.head()

,text,label
0,"Judgment on 18th April, 2008.Bhaskar J.These t...",0
1,"Beaumont, C.J.This is an appeal from a decisio...",0
2,The appellant has filed this appeal assailing ...,0
3,"The petitioner, Forbes Forbes Campbel and.Comp...",1
4,The writ petitioner was allotted plot No.60 fo...,1


In [10]:

test.head()

,text,label
0,Ratio Held Legal documents amend in the above ...,0
1,Counsel for Respondent - Ojha Honble Dr. Yeshw...,0
2,charge sheet has been filed in the present cas...,1
3,Coram Honble Ramesh C.J.Honble Alok Kumar Verm...,0
4,The has this appeal against the order dt.28th ...,0


In [33]:

import torch

print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU


In [34]:
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

MODEL_NAME = "law-ai/InLegalBERT"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.to(device)
model.eval()

print("Loaded InLegalBERT.")

Using device: cuda
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU
Loaded InLegalBERT.


In [13]:
def filter_max_tokens(df, column="text", max_len=512, batch_size=512):
    texts = df[column].astype(str).tolist()
    token_counts = []

    # batched tokenization
    for i in tqdm(range(0, len(texts), batch_size), desc="Tokenizing"):
        batch = texts[i:i+batch_size]
        enc = tokenizer(
            batch,
            add_special_tokens=True,
            truncation=False,
            padding=False
        )
        batch_lens = [len(ids) for ids in enc["input_ids"]]
        token_counts.extend(batch_lens)

    df["num_tokens"] = token_counts

    # filter rows <= max_len
    before = len(df)
    df = df[df["num_tokens"] <= max_len].reset_index(drop=True)
    after = len(df)

    print(f"Filtered: {before - after} rows removed (>{max_len} tokens). Remaining: {after}")

    return df

# ------------------------------------------------------
# 3) Apply to train, val, and test
# ------------------------------------------------------
print("\nProcessing TRAIN")
train = filter_max_tokens(train, column="text", max_len=512)

print("\nProcessing VAL")
val = filter_max_tokens(val, column="text", max_len=512)

print("\nProcessing TEST")
test = filter_max_tokens(test, column="text", max_len=512)

# ------------------------------------------------------
# 4) Final counts
# ------------------------------------------------------
print("\n================ FINAL ROW COUNTS ================")
print(f"TRAIN rows: {len(train)}")
print(f"VAL rows:   {len(val)}")
print(f"TEST rows:  {len(test)}")
print("==================================================")


Processing TRAIN


Tokenizing: 100%|████████████████████████████████████████████████████████████████████| 589/589 [16:19<00:00,  1.66s/it]


Filtered: 245780 rows removed (>512 tokens). Remaining: 55279

Processing VAL


Tokenizing: 100%|████████████████████████████████████████████████████████████████████| 169/169 [04:23<00:00,  1.56s/it]


Filtered: 70420 rows removed (>512 tokens). Remaining: 15597

Processing TEST


Tokenizing: 100%|██████████████████████████████████████████████████████████████████████| 84/84 [02:02<00:00,  1.46s/it]


Filtered: 35111 rows removed (>512 tokens). Remaining: 7897

================ FINAL ROW COUNTS ================
TRAIN rows: 55279
VAL rows:   15597
TEST rows:  7897


In [14]:
# 2) helper to batch-tokenize and count tokens
def count_tokens_in_df(df, text_col="text", batch_size=512, add_column=True):
    """
    Returns a list of token counts for each row in df[text_col].
    If add_column=True, adds a 'num_tokens' column to df (inplace).
    """
    texts = df[text_col].astype(str).tolist()
    n = len(texts)
    token_counts = [0] * n

    # process in batches to avoid huge memory peaks
    for i in tqdm(range(0, n, batch_size), desc="Tokenizing batches"):
        batch_texts = texts[i : i + batch_size]
        # tokenize without padding (we only need lengths); disable truncation so you see full length
        enc = tokenizer(batch_texts, add_special_tokens=True, truncation=False, padding=False)
        batch_lengths = [len(ids) for ids in enc["input_ids"]]
        token_counts[i : i + batch_size] = batch_lengths
    return token_counts

# 3) Run it on your datasets
# (Assumes `train`, `val`, `test` are pandas DataFrames already in the notebook)
for name, df in [("train", train), ("val", val), ("test", test)]:
    print(f"\n-> Processing {name} (rows: {len(df)})")
    count_tokens_in_df(df, text_col="text", batch_size=512, add_column=True)

    # quick stats
    print(f"{name} token stats:")
    print("  min:", int(df["num_tokens"].min()))
    print("  max:", int(df["num_tokens"].max()))
    print("  mean:", float(df["num_tokens"].mean()))
    print("  median:", float(df["num_tokens"].median()))
    print("  90th percentile:", int(df["num_tokens"].quantile(0.9)))

# 4) Optionally inspect top-longest examples
def show_longest(df, k=5):
    print(df.sort_values("num_tokens", ascending=False)[["text", "num_tokens"]].head(k).to_string(index=False))

print("\nTop longest in train:")
show_longest(train, k=5)


-> Processing train (rows: 55279)


Tokenizing batches: 100%|████████████████████████████████████████████████████████████| 108/108 [00:10<00:00, 10.17it/s]


train token stats:
  min: 55
  max: 512
  mean: 286.48937209428533
  median: 284.0
  90th percentile: 462

-> Processing val (rows: 15597)


Tokenizing batches: 100%|██████████████████████████████████████████████████████████████| 31/31 [00:02<00:00, 11.95it/s]


val token stats:
  min: 56
  max: 512
  mean: 286.5154837468744
  median: 284.0
  90th percentile: 462

-> Processing test (rows: 7897)


Tokenizing batches: 100%|██████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 11.75it/s]


test token stats:
  min: 58
  max: 512
  mean: 283.8234772698493
  median: 280.0
  90th percentile: 461

Top longest in train:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [15]:
train.head()

,text,label,num_tokens
0,Heard the learned counsel for the petitioner a...,0,225
1,All the above mentioned contempt petitions sha...,0,239
2,MR. JUSTICE AKHTAR HUSAIN KHAN (ORAL) Present ...,0,415
3,"THE HONBLE JUSTICE SMT.PRAKASH Anjana Prakash,...",0,361
4,To question the correctness of the order dated...,0,132


In [16]:
def extract_cls_embeddings_from_texts(
    texts,
    tokenizer,
    model,
    device,
    batch_size=32,
    max_length=512,
    show_progress=True
):
    """
    Given a list of texts, returns numpy array (n_texts, hidden_size) containing CLS embeddings.
    Uses outputs.last_hidden_state[:, 0, :] as CLS.
    """
    model.eval()
    all_embs = []
    n = len(texts)
    iterator = range(0, n, batch_size)
    if show_progress:
        iterator = tqdm(iterator, desc="Extracting CLS embeddings", unit="batch")

    try:
        for start in iterator:
            batch_texts = texts[start : start + batch_size]
            enc = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=max_length,
                return_tensors="pt",
                return_attention_mask=True
            )
            input_ids = enc["input_ids"].to(device)
            attention_mask = enc["attention_mask"].to(device)

            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                cls = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # (b, hidden)
                all_embs.append(cls)

        all_embs = np.vstack(all_embs)
        return all_embs

    except RuntimeError as e:
        # Likely OOM
        err = str(e).lower()
        if "out of memory" in err or "cuda" in err:
            print("\nRuntimeError (likely OOM). Try reducing batch_size or use CPU.")
            print("Suggested actions:")
            print(" - set batch_size to a smaller value (e.g., 8 or 4).")
            print(" - or run on CPU: device = torch.device('cpu') and move model to CPU.")
        raise

# ----- 3) Run extraction for train/val/test -----
# Choose a safe default batch size. If you have a big GPU, increase it.
DEFAULT_BATCH_SIZE = 32

print("\nExtracting TRAIN embeddings...")
train_texts = train["text"].astype(str).tolist()
X_train_cls = extract_cls_embeddings_from_texts(
    train_texts, tokenizer, model, device, batch_size=DEFAULT_BATCH_SIZE, max_length=512
)
print("X_train_cls shape:", X_train_cls.shape)

print("\nExtracting VAL embeddings...")
val_texts = val["text"].astype(str).tolist()
X_val_cls = extract_cls_embeddings_from_texts(
    val_texts, tokenizer, model, device, batch_size=DEFAULT_BATCH_SIZE, max_length=512
)
print("X_val_cls shape:", X_val_cls.shape)

print("\nExtracting TEST embeddings...")
test_texts = test["text"].astype(str).tolist()
X_test_cls = extract_cls_embeddings_from_texts(
    test_texts, tokenizer, model, device, batch_size=DEFAULT_BATCH_SIZE, max_length=512
)
print("X_test_cls shape:", X_test_cls.shape)

# ----- 4) Quick sanity previews -----
print("\nSanity checks:")
print("Train labels counts:", train['label'].value_counts().to_dict())
print("Shapes (train/val/test):", X_train_cls.shape, X_val_cls.shape, X_test_cls.shape)
print("Embedding dtype:", X_train_cls.dtype)
print("First train embedding (vector preview, first 6 elements):")
print(X_train_cls[0, :6])




Extracting TRAIN embeddings...


Extracting CLS embeddings: 100%|████████████████████████████████████████████████| 1728/1728 [22:54<00:00,  1.26batch/s]


X_train_cls shape: (55279, 768)

Extracting VAL embeddings...


Extracting CLS embeddings: 100%|██████████████████████████████████████████████████| 488/488 [06:22<00:00,  1.27batch/s]


X_val_cls shape: (15597, 768)

Extracting TEST embeddings...


Extracting CLS embeddings: 100%|██████████████████████████████████████████████████| 247/247 [02:36<00:00,  1.58batch/s]

X_test_cls shape: (7897, 768)

Sanity checks:
Train labels counts: {0: 28666, 1: 26613}
Shapes (train/val/test): (55279, 768) (15597, 768) (7897, 768)
Embedding dtype: float32
First train embedding (vector preview, first 6 elements):
[-0.10805389 -0.03798876  0.34646943 -0.33456892  0.6474604   0.17861015]


In [17]:
# ----- 5) Optionally save embeddings to disk (uncomment to enable) -----
out_dir = "inlegalbert_cls_embeddings"
os.makedirs(out_dir, exist_ok=True)
np.save(os.path.join(out_dir, "X_train_cls.npy"), X_train_cls)
np.save(os.path.join(out_dir, "X_val_cls.npy"), X_val_cls)
np.save(os.path.join(out_dir, "X_test_cls.npy"), X_test_cls)
print(f"\nSaved embeddings to folder: {out_dir} (files: X_train_cls.npy, X_val_cls.npy, X_test_cls.npy)")



Saved embeddings to folder: inlegalbert_cls_embeddings (files: X_train_cls.npy, X_val_cls.npy, X_test_cls.npy)


In [18]:
!pip install scikit-learn



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/8.9 MB 2.0 MB/s eta 0:00:05
      --------------------------------------- 0.2/8.9 MB 2.5 MB/s eta 0:00:04
     - -------------------------------------- 0.4/8.9 MB 3.1 MB/s eta 0:00:03
     -- ------------------------------------- 0.5/8.9 MB 3.4 MB/s eta 0:00:03
     --- ------------------------------------ 0.7/8.9 MB 3.5 MB/s eta 0:00:03
     ---- ----------------------------------- 0.9/8.9 MB 3.7 MB/s eta 0:00:03
     ----- ---------------------------------- 1.1/8.9 MB 3.8 MB/s eta 0:00:03
     ------ --------------------------------- 1.4/8.9 MB 4.1 MB/s eta 0:00:02
     ------ --------------------------------- 1.5/8.9 MB 4.1 MB/s eta 0:00:02
     -------- ------------------------------- 1.8/8.9 MB 4.2 MB/s eta 0:00:02
     -------- ------------------------------- 1.9/8.9 MB 4.2 MB/s eta 0:00:02
     --------- ------------------------------ 2.2/8.9 MB 4.3 MB/s eta 

In [19]:

from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import joblib

In [20]:
!pip install xgboost


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
     ---------------------------------------- 0.1/72.0 MB 2.6 MB/s eta 0:00:28
     ---------------------------------------- 0.2/72.0 MB 3.5 MB/s eta 0:00:21
     ---------------------------------------- 0.5/72.0 MB 4.3 MB/s eta 0:00:17
     ---------------------------------------- 0.6/72.0 MB 4.7 MB/s eta 0:00:16
     ---------------------------------------- 0.8/72.0 MB 4.5 MB/s eta 0:00:16
      --------------------------------------- 1.0/72.0 MB 4.5 MB/s eta 0:00:16
      --------------------------------------- 1.1/72.0 MB 4.2 MB/s eta 0:00:17
      --------------------------------------- 1.4/72.0 MB 4.5 MB/s eta 0:00:16
      --------------------------------------- 1.5/72.0 MB 4.6 MB/s eta 0:00:16
      --------------------------------------- 1.8/72.0 MB 4.7 MB/s eta 0:00:16
     - -------------------------------------- 1.9/72.0 MB 4.8 MB/s eta 0:00:15
     - -------------------------------------- 2.1/72.0 MB 

In [21]:
try:
    X_train_cls  # noqa: F821
    X_val_cls
    X_test_cls
    print("Embeddings found in memory.")
except NameError:
    print("Embeddings not found in memory — loading from inlegalbert_cls_embeddings/*.npy")
    X_train_cls = np.load("inlegalbert_cls_embeddings/X_train_cls.npy")
    X_val_cls = np.load("inlegalbert_cls_embeddings/X_val_cls.npy")
    X_test_cls = np.load("inlegalbert_cls_embeddings/X_test_cls.npy")

# Labels
y_train = train["label"].to_numpy()
y_val = val["label"].to_numpy()
y_test = test["label"].to_numpy()

print("Shapes (train/val/test):", X_train_cls.shape, X_val_cls.shape, X_test_cls.shape)
print("Label counts (train):", np.bincount(y_train))
print("Label counts (val):", np.bincount(y_val))
print("Label counts (test):", np.bincount(y_test))


Embeddings found in memory.
Shapes (train/val/test): (55279, 768) (15597, 768) (7897, 768)
Label counts (train): [28666 26613]
Label counts (val): [8163 7434]
Label counts (test): [4098 3799]


In [22]:
# --------- 1) Scale features ----------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_cls)
X_val_scaled = scaler.transform(X_val_cls)
X_test_scaled = scaler.transform(X_test_cls)



In [23]:
# --------- 2) Grid search for LinearSVC ----------
param_grid = {"C": [0.01, 0.1, 1.0, 10.0]}
base_svc = LinearSVC(max_iter=10000, tol=1e-4)   # increase max_iter if you get convergence warnings

grid = GridSearchCV(
    estimator=base_svc,
    param_grid=param_grid,
    cv=3,
    scoring="f1_macro",
    n_jobs=-1,
    verbose=2
)

print("\nStarting GridSearchCV for LinearSVC (this may take a few minutes)...")
grid.fit(X_train_scaled, y_train)

print("\nBest params:", grid.best_params_)
best_svc = grid.best_estimator_


Starting GridSearchCV for LinearSVC (this may take a few minutes)...
Fitting 3 folds for each of 4 candidates, totalling 12 fits

Best params: {'C': 0.01}


In [24]:
# --------- 3) Evaluate helper ----------
def evaluate_model(model, X, y, split_name="VAL"):
    y_pred = model.predict(X)
    acc = accuracy_score(y, y_pred)
    f1_macro = f1_score(y, y_pred, average="macro")
    print(f"\n--- {split_name} results ---")
    print("Accuracy:", acc)
    print("Macro F1:", f1_macro)
    print("Classification report:")
    print(classification_report(y, y_pred))
    print("Confusion matrix:")
    print(confusion_matrix(y, y_pred))
    return {"accuracy": acc, "f1_macro": f1_macro}

In [25]:
# Evaluate on validation set
val_metrics = evaluate_model(best_svc, X_val_scaled, y_val, split_name="VAL")

# Evaluate on test set
test_metrics = evaluate_model(best_svc, X_test_scaled, y_test, split_name="TEST")



--- VAL results ---
Accuracy: 0.6974418157337949
Macro F1: 0.6958566562880777
Classification report:
              precision    recall  f1-score   support

           0       0.70      0.74      0.72      8163
           1       0.69      0.66      0.67      7434

    accuracy                           0.70     15597
   macro avg       0.70      0.70      0.70     15597
weighted avg       0.70      0.70      0.70     15597

Confusion matrix:
[[6002 2161]
 [2558 4876]]

--- TEST results ---
Accuracy: 0.7021653792579461
Macro F1: 0.7011201631294279
Classification report:
              precision    recall  f1-score   support

           0       0.70      0.73      0.72      4098
           1       0.70      0.67      0.68      3799

    accuracy                           0.70      7897
   macro avg       0.70      0.70      0.70      7897
weighted avg       0.70      0.70      0.70      7897

Confusion matrix:
[[3006 1092]
 [1260 2539]]


In [26]:


# --------- 4) Save artifacts ----------
out_dir = "inlegalbert_svm_model"
os.makedirs(out_dir, exist_ok=True)
joblib.dump(best_svc, os.path.join(out_dir, "linear_svc_best.joblib"))
joblib.dump(scaler, os.path.join(out_dir, "scaler.joblib"))
print(f"\nSaved model and scaler to {out_dir}")

# --------- 5) Optional: show top-level summary ----------
print("\nSUMMARY:")
print("Best C:", grid.best_params_["C"])
print("VAL metrics:", val_metrics)
print("TEST metrics:", test_metrics)


Saved model and scaler to inlegalbert_svm_model

SUMMARY:
Best C: 0.01
VAL metrics: {'accuracy': 0.6974418157337949, 'f1_macro': 0.6958566562880777}
TEST metrics: {'accuracy': 0.7021653792579461, 'f1_macro': 0.7011201631294279}


In [27]:
# ------------------ 0) load embeddings & labels if needed ------------------
try:
    X_train_cls  # noqa
    X_val_cls
    X_test_cls
    print("Embeddings found in memory.")
except NameError:
    print("Loading embeddings from disk...")
    X_train_cls = np.load("inlegalbert_cls_embeddings/X_train_cls.npy")
    X_val_cls = np.load("inlegalbert_cls_embeddings/X_val_cls.npy")
    X_test_cls = np.load("inlegalbert_cls_embeddings/X_test_cls.npy")

y_train = train["label"].to_numpy()
y_val = val["label"].to_numpy()
y_test = test["label"].to_numpy()

print("Shapes:", X_train_cls.shape, X_val_cls.shape, X_test_cls.shape)

# ------------------ 1) Prepare scaled versions (for logistic) ------------------
scaler_log = StandardScaler()
X_train_log = scaler_log.fit_transform(X_train_cls)
X_val_log = scaler_log.transform(X_val_cls)
X_test_log = scaler_log.transform(X_test_cls)

# Save scaler if needed
os.makedirs("models_compare", exist_ok=True)
joblib.dump(scaler_log, "models_compare/scaler_log.joblib")

# ------------------ 2) Load existing SVM test f1 (if available) ------------------
best_existing_f1 = None
svm_path = "inlegalbert_svm_model/linear_svc_best.joblib"
if os.path.exists(svm_path):
    try:
        # We don't have stored metrics file; if you saved them, load; else we can re-evaluate SVM
        existing_svc = joblib.load(svm_path)
        # Scale test data using saved scaler if exists
        scaler_path = "inlegalbert_svm_model/scaler.joblib"
        if os.path.exists(scaler_path):
            existing_scaler = joblib.load(scaler_path)
            X_test_for_svm = existing_scaler.transform(X_test_cls)
        else:
            X_test_for_svm = X_test_log  # fallback
        y_pred_svm = existing_svc.predict(X_test_for_svm)
        best_existing_f1 = f1_score(y_test, y_pred_svm, average="macro")
        print(f"Existing saved SVM test macro-F1: {best_existing_f1:.4f}")
    except Exception:
        best_existing_f1 = None








Embeddings found in memory.
Shapes: (55279, 768) (15597, 768) (7897, 768)
Existing saved SVM test macro-F1: 0.7011


In [28]:
# Utility to fit, evaluate and store best model
def fit_and_eval(clf, Xtr, ytr, Xval, yval, Xte, yte, searcher, model_name):
    t0 = time()
    searcher.fit(Xtr, ytr)
    elapsed = time() - t0
    best = searcher.best_estimator_
    print(f"\n{model_name} best params: {searcher.best_params_} (fit time: {elapsed:.1f}s)")
    # Evaluate on val & test
    yval_pred = best.predict(Xval)
    ytest_pred = best.predict(Xte)
    val_f1 = f1_score(yval, yval_pred, average="macro")
    test_f1 = f1_score(yte, ytest_pred, average="macro")
    print(f"{model_name} VAL macro-F1: {val_f1:.4f} | TEST macro-F1: {test_f1:.4f}")
    print("Val classification report:")
    print(classification_report(yval, yval_pred))
    print("Test classification report:")
    print(classification_report(yte, ytest_pred))
    # store
    results[model_name] = {
        "best_estimator": best,
        "best_params": searcher.best_params_,
        "val_f1": val_f1,
        "test_f1": test_f1,
        "fit_time_s": elapsed
    }
    return best

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import warnings
from time import time
warnings.filterwarnings("ignore")
# ------------------ 3) Define models and search grids ------------------

results = {}

# 3.a Logistic Regression (GridSearch)
lr = LogisticRegression(max_iter=2000, solver="saga", random_state=42)
lr_param_grid = {
    "C": [0.01, 0.1, 1.0, 10.0],
    "penalty": ["l2"],  # saga supports l1/l2/elasticnet - l2 is robust; add l1 if you want sparsity
}

# 3.b Random Forest (GridSearch)
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_param_grid = {
    "n_estimators": [200, 500],
    "max_depth": [None, 20, 50],
    "min_samples_split": [2, 5],
}

# 3.c XGBoost (RandomizedSearch for speed)
try:
    from xgboost import XGBClassifier
    xgb_available = True
except Exception:
    xgb_available = False
    print("xgboost not available; skipping XGBoost. Install with `pip install xgboost` to enable.")

if xgb_available:
    xgb = XGBClassifier(objective="binary:logistic", use_label_encoder=False, eval_metric="logloss", tree_method="auto", random_state=42)
    xgb_param_dist = {
        "n_estimators": [100, 200, 400],
        "max_depth": [3, 6, 10],
        "learning_rate": [0.01, 0.05, 0.1],
        "subsample": [0.6, 0.8, 1.0],
        "colsample_bytree": [0.6, 0.8, 1.0],
        "reg_alpha": [0, 0.1, 1],
        "reg_lambda": [1, 5, 10],}

In [35]:
# ------------------ 4) Run GridSearch / RandomizedSearch ------------------
# NOTE: adjust cv and n_iter for speed if needed
cv_folds = 3

# Logistic Regression
print("\n>> Tuning Logistic Regression (on scaled embeddings)...")
lr_search = GridSearchCV(lr, lr_param_grid, scoring="f1_macro", cv=cv_folds, n_jobs=-1, verbose=2)
best_lr = fit_and_eval(lr, X_train_log, y_train, X_val_log, y_val, X_test_log, y_test, lr_search, "LogisticRegression")

# Random Forest
print("\n>> Tuning Random Forest (on raw embeddings)...")
rf_search = GridSearchCV(rf, rf_param_grid, scoring="f1_macro", cv=cv_folds, n_jobs=-1, verbose=2)
best_rf = fit_and_eval(rf, X_train_cls, y_train, X_val_cls, y_val, X_test_cls, y_test, rf_search, "RandomForest")

# XGBoost (if available) - use RandomizedSearchCV for speed
best_xgb = None
if xgb_available:
    from sklearn.model_selection import RandomizedSearchCV
    print("\n>> Tuning XGBoost (on raw embeddings) with RandomizedSearchCV...")
    xgb_search = RandomizedSearchCV(xgb, xgb_param_dist, n_iter=20, scoring="f1_macro", cv=cv_folds, n_jobs=1, verbose=2, random_state=42)
    best_xgb = fit_and_eval(xgb, X_train_cls, y_train, X_val_cls, y_val, X_test_cls, y_test, xgb_search, "XGBoost")



>> Tuning Logistic Regression (on scaled embeddings)...
Fitting 3 folds for each of 4 candidates, totalling 12 fits

LogisticRegression best params: {'C': 0.01, 'penalty': 'l2'} (fit time: 1203.8s)
LogisticRegression VAL macro-F1: 0.6968 | TEST macro-F1: 0.7024
Val classification report:
              precision    recall  f1-score   support

           0       0.70      0.74      0.72      8163
           1       0.69      0.66      0.67      7434

    accuracy                           0.70     15597
   macro avg       0.70      0.70      0.70     15597
weighted avg       0.70      0.70      0.70     15597

Test classification report:
              precision    recall  f1-score   support

           0       0.71      0.74      0.72      4098
           1       0.70      0.67      0.68      3799

    accuracy                           0.70      7897
   macro avg       0.70      0.70      0.70      7897
weighted avg       0.70      0.70      0.70      7897


>> Tuning Random Forest (on

In [36]:
test_metrics = evaluate_model(best_svc, X_test_scaled, y_test, split_name="TEST")
# --------- 6) Add SVM results to global results dict ----------
model_name = "LinearSVM"

results[model_name] = {
    "best_estimator": best_svc,
    "best_params": grid.best_params_,
    "val_f1": val_metrics["f1_macro"],
    "test_f1": test_metrics["f1_macro"],
    "fit_time_s": grid.cv_results_["mean_fit_time"][grid.best_index_]
}

print("\nAdded LinearSVM to results dict.")
print(results[model_name])



--- TEST results ---
Accuracy: 0.7021653792579461
Macro F1: 0.7011201631294279
Classification report:
              precision    recall  f1-score   support

           0       0.70      0.73      0.72      4098
           1       0.70      0.67      0.68      3799

    accuracy                           0.70      7897
   macro avg       0.70      0.70      0.70      7897
weighted avg       0.70      0.70      0.70      7897

Confusion matrix:
[[3006 1092]
 [1260 2539]]

Added LinearSVM to results dict.
{'best_estimator': LinearSVC(C=0.01, max_iter=10000), 'best_params': {'C': 0.01}, 'val_f1': 0.6958566562880777, 'test_f1': 0.7011201631294279, 'fit_time_s': np.float64(96.30051652590434)}


In [37]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier

# ------------------ NEW ENSEMBLE (works for all models) ------------------

ensemble_estimators = []

for model_name, info in results.items():
    est = info["best_estimator"]
    
    # If model was trained on scaled embeddings
    if "scaler" in info:
        pipe = Pipeline([
            ("scaler", info["scaler"]),
            ("clf", est)
        ])
        ensemble_estimators.append((model_name, pipe))
    else:
        # raw-embedding models (e.g., RandomForest, XGBoost)
        ensemble_estimators.append((model_name, est))


if len(ensemble_estimators) >= 2:
    # build ensemble with all models
    voting = VotingClassifier(
        estimators=ensemble_estimators,
        voting="hard",
        n_jobs=-1
    )
    
    print("\nTraining VotingClassifier using ALL models...")
    voting.fit(X_train_cls, y_train)

    # Predict
    y_pred = voting.predict(X_test_cls)
    ensemble_f1 = f1_score(y_test, y_pred, average="macro")
    print("Ensemble TEST macro-F1:", ensemble_f1)

    # Save
    joblib.dump(voting, "models_compare/voting_ensemble_hard.joblib")
    print("Saved ensemble to models_compare/voting_ensemble_hard.joblib")

print("\nDone. Ensemble built using all models in results.")



Training VotingClassifier using ALL models...
Ensemble TEST macro-F1: 0.7305721555926556
Saved ensemble to models_compare/voting_ensemble_hard.joblib

Done. Ensemble built using all models in results.


In [38]:
import joblib

joblib.dump(results, "models_compare/results_dict.joblib")
print("Saved results_dict.joblib ✅")
joblib.dump(X_train_cls, "models_compare/X_train_cls.joblib")
joblib.dump(y_train, "models_compare/y_train.joblib")
joblib.dump(X_test_cls, "models_compare/X_test_cls.joblib")
joblib.dump(y_test, "models_compare/y_test.joblib")

print("Saved train/test embeddings and labels ✅")


Saved results_dict.joblib ✅
Saved train/test embeddings and labels ✅


In [39]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.base import clone
import numpy as np
import joblib

# =========================================================
# 1. Build final estimators list with correct scaling
# =========================================================

stack_estimators = []

for model_name, info in results.items():
    est = info["best_estimator"]
    
    # wrap model in pipeline if it used a scaler
    if "scaler" in info and info["scaler"] is not None:
        pipe = Pipeline([
            ("scaler", info["scaler"]),
            ("clf", est)
        ])
        stack_estimators.append((model_name, pipe))
    else:
        stack_estimators.append((model_name, est))

print("\nBase estimators for stacking:", [name for name, _ in stack_estimators])

# =========================================================
# 2. Out-of-fold prediction matrix for stacking
# =========================================================

n_models = len(stack_estimators)
n_train = X_train_cls.shape[0]
n_test = X_test_cls.shape[0]
n_classes = len(np.unique(y_train))

# OOF predictions: (n_samples, n_models * n_classes)
oof_train = np.zeros((n_train, n_models * n_classes))
test_pred_accumulator = np.zeros((n_test, n_models * n_classes))

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("\nGenerating OOF predictions...")

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_cls, y_train)):
    print(f"  Fold {fold+1}/5")
    
    X_tr, X_val = X_train_cls[train_idx], X_train_cls[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    
    col = 0
    for name, model in stack_estimators:

        cloned_model = clone(model)
        cloned_model.fit(X_tr, y_tr)
        
        # -------- Unified prediction function --------
        def get_proba(m, X):
            if hasattr(m, "predict_proba"):
                return m.predict_proba(X)
            elif hasattr(m, "decision_function"):
                df = m.decision_function(X)
                # Handle binary classifiers
                if df.ndim == 1:
                    df = np.vstack([-df, df]).T
                exp = np.exp(df - df.max(axis=1, keepdims=True))
                return exp / exp.sum(axis=1, keepdims=True)
            else:
                preds = m.predict(X)
                oh = np.zeros((len(preds), n_classes))
                oh[np.arange(len(preds)), preds] = 1
                return oh

        # OOF validation predictions
        val_proba = get_proba(cloned_model, X_val)
        oof_train[val_idx, col:col+n_classes] = val_proba
        
        # Test predictions
        test_proba = get_proba(cloned_model, X_test_cls)
        test_pred_accumulator[:, col:col+n_classes] += test_proba
        
        col += n_classes

# average test predictions
oof_test = test_pred_accumulator / skf.n_splits

print("OOF matrix shape:", oof_train.shape)
print("Test stacked matrix shape:", oof_test.shape)

# =========================================================
# 3. Train meta-model on OOF predictions
# =========================================================

meta_clf = LogisticRegression(max_iter=5000)
meta_clf.fit(oof_train, y_train)

# evaluate performance
y_test_pred = meta_clf.predict(oof_test)
stack_test_f1 = f1_score(y_test, y_test_pred, average="macro")

print("\n=== STACKING RESULTS ===")
print("Stacking Test Macro F1:", stack_test_f1)

# =========================================================
# 4. Save stacking components
# =========================================================

joblib.dump(stack_estimators, "models_compare/stack_base_models.joblib")
joblib.dump(meta_clf, "models_compare/stack_meta_clf.joblib")

print("Saved stack_base_models.joblib and stack_meta_clf.joblib")



Base estimators for stacking: ['LogisticRegression', 'RandomForest', 'XGBoost', 'LinearSVM']

Generating OOF predictions...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
OOF matrix shape: (55279, 8)
Test stacked matrix shape: (7897, 8)

=== STACKING RESULTS ===
Stacking Test Macro F1: 0.7619124776815973
Saved stack_base_models.joblib and stack_meta_clf.joblib


In [40]:
from sklearn.metrics import classification_report

# -----------------------------
# 1. Stacking Model Report
# -----------------------------
print("=== Stacking Classifier Report ===")
stack_report = classification_report(y_test, y_test_pred)
print(stack_report)


=== Stacking Classifier Report ===
              precision    recall  f1-score   support

           0       0.75      0.81      0.78      4098
           1       0.77      0.72      0.74      3799

    accuracy                           0.76      7897
   macro avg       0.76      0.76      0.76      7897
weighted avg       0.76      0.76      0.76      7897



In [41]:
print("=== Voting Classifier Report ===")
voting_report = classification_report(y_test, y_pred)
print(voting_report)

=== Voting Classifier Report ===
              precision    recall  f1-score   support

           0       0.70      0.85      0.77      4098
           1       0.79      0.61      0.69      3799

    accuracy                           0.74      7897
   macro avg       0.75      0.73      0.73      7897
weighted avg       0.75      0.74      0.73      7897

